<a href="https://colab.research.google.com/github/BulusuSrikar/DL-Assignment-2/blob/main/Exp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install --upgrade accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [4]:
from datasets import Dataset
import pandas as pd

# Load your lyrics text file
with open("/content/Eminem.csv", "r", encoding="utf-8") as f:
    lines = f.read().split("\n\n")  # separate songs

df = pd.DataFrame({"text": lines})
dataset = Dataset.from_pandas(df)

In [5]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [7]:
import os
os.environ["WANDB_MODE"] = "disabled"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-lyrics",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="no",
    report_to="none",  # <- disables W&B, TensorBoard, etc.
)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: professionalonly12345 (professionalonly12345-microsoft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=3, training_loss=4.402717272440593, metrics={'train_runtime': 372.5531, 'train_samples_per_second': 0.008, 'train_steps_per_second': 0.008, 'total_flos': 783876096000.0, 'train_loss': 4.402717272440593, 'epoch': 3.0})

In [12]:
from transformers import pipeline, set_seed

# Use the fine-tuned model and tokenizer if you've already loaded them
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Set a seed for reproducibility
set_seed(42)

# Generate lyrics based on a prompt (e.g., Juice WRLD, Rap God)
lyrics = generator("The Real Slim Shady", max_length=100, num_return_sequences=1)

# Print the generated lyrics
print(lyrics[0]['generated_text'])

generator("The Real Slim Shady", max_length=100, num_return_sequences=1, temperature=0.8, top_k=50)



Device set to use cuda:0


The Real Slim Shady Jacket & Pocket Eye Lipsticks (C.S.P.P.)

Matching Systems:

These are my top rated line of shoes

W

The C.S.P.P.

Mostly

Mostly

Satisfying Measurements:

I put on a couple of ICPs (I, A, & G):

A

E
E-X-A




[{'generated_text': 'The Real Slim Shady and the Big Secret, by Tim Schafer\n\n\nThe first thing that came to my mind was that the name "Tripod" was a nickname that some would call "Dinner" because I am a big fan of that. "Dinner" is a pretty cool descriptor, but it\'s not a "Trip" for me. I\'m a big fan of the "Top 10" and many of the Top 10 are good albums and some are very good'}]